In [ ]:
from chatbots import OpenAiChatBot


chatbot = OpenAiChatBot()

chatbot.run()

In [ ]:
tools_functions: list[dict] = []

get_url_function_tool = {
    "name": "get_url_content",
    "description": "Find the content about of urls that the user gives. Call this whenever you need to know some content about some url.",
    "parameters": {
        "type": "object",
        "properties": {
            "url": {
                "type": "string",
                "description": "The url that the user wants to know something about",
            },
        },
        "require": ["url"],
        "additionalProperties": False
    }
}

tools_functions.append({"type": "function", "function": get_url_function_tool})
